## Used Libraries

In [1]:
import numpy as np
import pandas as pd
import re

## Important Helper Functions

In [2]:
def load_eurostat_tsv(path):
    """
    Load and clean a Eurostat-formatted TSV dataset.

    This function handles the specific structure used by Eurostat in `.tsv` files,
    where multiple metadata fields (e.g. freq, unit, geo) are embedded inside the
    first column separated by commas, and year values are spread across multiple
    wide-format columns.

    The function performs the following steps:
    - Reads the TSV file using tab separation.
    - Automatically detects and splits the composite metadata column into
      individual fields (e.g. 'freq', 'unit', 'c_resid', 'nace_r2', 'geo').
    - Renames Eurostat's encoded "geo\\TIME_PERIOD" field to a clean "geo".
    - Identifies year columns (four-digit names) and cleans their values:
        * Converts ":" into NaN.
        * Removes Eurostat suffixes ("e", "b", "p") indicating estimated,
          provisional, or series breaks.
        * Converts cleaned values to numeric.
    - Reorders columns so metadata fields appear first (geo, freq, c_resid,
      unit, nace_r2), followed by year columns.
    - Returns a tidy, analysis-ready DataFrame in wide format.

    Parameters
    ----------
    path : str
        Path to the Eurostat TSV file.

    Returns
    -------
    pandas.DataFrame
        A cleaned DataFrame where:
        - Metadata fields are split into separate columns.
        - The 'geo' column is properly extracted and placed first.
        - Year columns contain numeric values or NaN.
        - Non-numeric Eurostat artifacts and flags are removed.

    Notes
    -----
    - This function works with ANY Eurostat `.tsv` dataset based on the standard
      multi-dimension TSV export format (tourism, GDP, population density, etc.).
    - The output is kept in wide format for flexibility; use a melt/reshape
      helper for long-format needs.
    - Eurostat datasets often vary in the number of metadata fields included in
      the first column; this function dynamically adapts to those differences.
    """

    df = pd.read_csv(path, sep="\t")

    # 1. Detect and split the composite first column
    first_col = df.columns[0]
    meta_keys = first_col.split(",")

    df[meta_keys] = df[first_col].str.split(",", expand=True)
    df.drop(columns=[first_col], inplace=True)

    # 2. Rename geo\TIME_PERIOD → geo
    for col in df.columns:
        if col.startswith("geo"):
            df.rename(columns={col: "geo"}, inplace=True)
            break

    # 3. Clean year columns
    year_cols = [c for c in df.columns if re.match(r"^\d{4}$", str(c))]

    for col in year_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(":", np.nan)
            .str.replace(" e", "", regex=False)
            .str.replace(" b", "", regex=False)
            .str.replace(" p", "", regex=False)
        )

        df[col] = pd.to_numeric(df[col], errors="coerce")

    df.rename(columns={c: int(c) for c in year_cols}, inplace=True)

    # 4. Reorder: geo first
    meta_present = [c for c in ["geo", "freq", "c_resid", "unit", "nace_r2"] if c in df.columns]
    other_cols = [c for c in df.columns if c not in meta_present]

    df = df[meta_present + other_cols]

    return df

In [3]:
def eurostat_to_long(df, value_name):
    """
    Convert a cleaned Eurostat wide-format dataframe into long format.

    Parameters
    ----------
    df : pandas.DataFrame
        Output of load_eurostat_tsv()
    value_name : str
        Name of the metric column (e.g. 'nights_spent', 'gdp', 'pop_density')

    Returns
    -------
    pandas.DataFrame
        Long-format dataframe with columns: geo, year, <value_name>
    """

    # Identify year columns
    year_cols = [c for c in df.columns if isinstance(c, int)]

    # Melt
    long_df = df.melt(
        id_vars=["geo"],
        value_vars=year_cols,
        var_name="year",
        value_name=value_name
    )

    # Clean year column
    long_df["year"] = long_df["year"].astype(int)

    return long_df


In [4]:
def clean_year_columns(df):
    """
    Standardize Eurostat year column names.

    Cleans column names by removing spaces and Eurostat flags ('e', 'b', 'p'),
    and converts any 4-digit year strings (e.g. '2012', '2012 e') into integer
    column names. This ensures year columns are correctly detected when
    converting to long format.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame returned by load_eurostat_tsv().

    Returns
    -------
    pandas.DataFrame
        Same DataFrame with cleaned and integer-typed year columns.
    """
    fixed_cols = {}
    for col in df.columns:
        col_str = str(col).strip()              # remove spaces
        col_str = col_str.replace(" e", "")      # remove flags
        col_str = col_str.replace(" b", "")
        col_str = col_str.replace(" p", "")
        if col_str.isdigit() and len(col_str) == 4:
            fixed_cols[col] = int(col_str)
    df.rename(columns=fixed_cols, inplace=True)
    return df


## Tourist Dataset (target variable)

In [5]:
tourism_df = load_eurostat_tsv("raw_data/estat_tour_occ_nin2.tsv")
tourism_df

,geo,freq,c_resid,unit,nace_r2,1990,1991,1992,1993,1994,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,AL,A,DOM,NR,I551,:,:,:,:,:,...,:,:,:,804181 e,1015037 e,729659 e,1076608 e,1237862 e,1943039 e,2039675
1,AL0,A,DOM,NR,I551,:,:,:,:,:,...,:,:,:,804181 e,1015037 e,729659 e,1076608 e,1237862 e,1943039 e,2039675
2,AL01,A,DOM,NR,I551,:,:,:,:,:,...,:,:,:,199081 e,259910 e,148755 e,206227 e,261012 e,467356 e,501278
3,AL02,A,DOM,NR,I551,:,:,:,:,:,...,:,:,:,269005 e,329702 e,164554 e,245447 e,372722 e,514057 e,598752
4,AL03,A,DOM,NR,I551,:,:,:,:,:,...,:,:,:,336095 e,425425 e,416350 e,624934 e,604128 e,961626 e,939645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32259,UKM3,A,TOTAL,P_THAB,I551-I553,:,:,:,:,:,...,:,5324.41,:,:,:,:,:,:,:,:
32260,UKM5,A,TOTAL,P_THAB,I551-I553,:,:,:,:,:,...,8546.93,8436.02,:,:,:,:,:,:,:,:
32261,UKM6,A,TOTAL,P_THAB,I551-I553,:,:,:,:,:,...,10986.14,19337.05,:,:,:,:,:,:,:,:
32262,UKN,A,TOTAL,P_THAB,I551-I553,1443.35 e,:,:,:,1880,...,7341.56,7397.07,:,:,:,:,:,:,:,:


## GDP Dataset (feature variable)

In [6]:
gdp_df = load_eurostat_tsv("raw_data/estat_nama_10r_2gdp.tsv")
gdp_df

,geo,freq,unit,2000,2001,2002,2003,2004,2005,2006,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AL,A,EUR_HAB,:,:,:,:,:,:,:,...,3500,3600,3700,4000,4500,4800,4700,5400 p,:,:
1,AL0,A,EUR_HAB,:,:,:,:,:,:,:,...,3500,3600,3700,4000,4500,4800,4700,5400 p,:,:
2,AL01,A,EUR_HAB,:,:,:,:,:,:,:,...,2700,2900,3100,3300,3600,3900,3700,4300 p,:,:
3,AL02,A,EUR_HAB,:,:,:,:,:,:,:,...,4100,4300,4400,4900,5400,5800,5600,6500 p,:,:
4,AL03,A,EUR_HAB,:,:,:,:,:,:,:,...,3400,3300,3400,3600,4000,4400,4300,4900 p,:,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,TRB2,A,PPS_HAB_EU27_2020,:,:,:,:,15,16,17,...,25,26,26,26,25,24,25,22,25,28
3161,TRC,A,PPS_HAB_EU27_2020,:,:,:,:,21,21,22,...,33,34,33,33,31,29,32,31,36,38
3162,TRC1,A,PPS_HAB_EU27_2020,:,:,:,:,25,26,27,...,41,44,42,42,40,38,43,44,50,50
3163,TRC2,A,PPS_HAB_EU27_2020,:,:,:,:,20,20,21,...,27,28,27,27,24,23,24,22,26,29


## Population Density Dataset (feature variable)

In [7]:
pop_df = load_eurostat_tsv("raw_data/estat_tgs00024.tsv")
pop_df

,geo,freq,unit,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AL01,A,PER_KM2,79.2,78.5,77.7,77.0,76.6,76.1,75.5,74.8,77.0 e,76.3 e,74.0 e,:
1,AL02,A,PER_KM2,219.6,222.3,225.1,227.0,229.2,232.4,234.8,236.2,237.8 e,238.6 e,238.8 e,:
2,AL03,A,PER_KM2,73.3,72.4,71.6,70.7,69.9,68.9,67.9,67.1,68.8 e,67.9 e,64.9 e,:
3,AT11,A,PER_KM2,78.0,78.2,78.5,79.0,79.4 b,79.7,77.5,77.8,77.8,78.1,79.2,79.7
4,AT12,A,PER_KM2,85.5,85.7,86.2,87.0,87.7 b,88.2,88.5,88.9,88.9,89.3,90.4,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,UKM6,A,PER_KM2,11.5,11.6,11.6,11.6,11.6 b,11.6,11.7,:,:,:,:,:
334,UKM7,A,PER_KM2,145.2,145.9,146.8,147.8,149.0 b,150.0,150.8,:,:,:,:,:
335,UKM8,A,PER_KM2,778.5,779.7,782.1,786.4,793.1 b,797.2,800.0,:,:,:,:,:
336,UKM9,A,PER_KM2,59.6,59.6,59.5,59.6,59.7 b,59.7,59.7,:,:,:,:,:


## Cleaning year columns

In [8]:
tourism_df = clean_year_columns(tourism_df)
gdp_df = clean_year_columns(gdp_df)
pop_df = clean_year_columns(pop_df)


## Converting Dataframes to Long Format

In [9]:
tourism_long = eurostat_to_long(tourism_df, value_name="nights_spent")
gdp_long = eurostat_to_long(gdp_df, value_name="gdp")
pop_long = eurostat_to_long(pop_df, value_name="pop_density")


## Merging Datasets

In [10]:
df = tourism_long.merge(gdp_long, on=["geo", "year"], how="inner")
df = df.merge(pop_long, on=["geo", "year"], how="inner")

target = "nights_spent"

df = df[[c for c in df.columns if c != target] + [target]]

df

,geo,year,gdp,pop_density,nights_spent
0,AL01,2012,2700,79.2,:
1,AL01,2012,10,79.2,:
2,AL01,2012,2297.46,79.2,:
3,AL01,2012,319439.28,79.2,:
4,AL01,2012,5402.82,79.2,:
...,...,...,...,...,...
823699,TRC3,2023,15193.67,94.2,395.78
823700,TRC3,2023,391384.46,94.2,395.78
823701,TRC3,2023,34548.05,94.2,395.78
823702,TRC3,2023,14300,94.2,395.78
